<a href="https://colab.research.google.com/github/charlesjdf/AI-FINALS/blob/main/CNN_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [74]:
# Google Drive mount
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [75]:
# Libraries
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

In [76]:
# directory paths for training and validation data
TRAIN_DIR = '/content/drive/MyDrive/AI/datasets/Meat_Freshness.v1-new-dataset.multiclass/train'
VAL_DIR = '/content/drive/MyDrive/AI/datasets/Meat_Freshness.v1-new-dataset.multiclass/valid'
IMG_SIZE = (208, 208)
BATCH_SIZE = 115

In [77]:
# Data Preparation
def img_generators(train_dir, val_dir, img_size=IMG_SIZE, batch_size=BATCH_SIZE):
      # Check if the directories exist and contain data
    if not os.path.exists(train_dir) or len(os.listdir(train_dir)) == 0:
        raise ValueError(f"Training directory '{train_dir}' does not exist or is empty.")
    if not os.path.exists(val_dir) or len(os.listdir(val_dir)) == 0:
        raise ValueError(f"Validation directory '{val_dir}' does not exist or is empty.")

    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True
    )
    val_datagen = ImageDataGenerator(rescale=1./255)

    train_generator = train_datagen.flow_from_directory(
        TRAIN_DIR,
        target_size=img_size,
        batch_size=batch_size,
        class_mode='binary'
    )

    val_generator = val_datagen.flow_from_directory(
        val_dir,
        target_size=img_size,
        batch_size=batch_size,
        class_mode='binary'
    )

    return train_generator, val_generator

In [78]:
# CNN Model
def cnn_model(input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D(2, 2),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')  # Binary classification
    ])

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model


In [79]:
# Model Training
def train_cnn_model(model, train_generator, val_generator, epochs=10):
    early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
    history = model.fit(
        train_generator,
        validation_data=val_generator,
        epochs=epochs,
        callbacks=[early_stopping]
    )
    return model, history

In [80]:
# Main function to load data, build, and train the model
def main():
    train_generator, val_generator = img_generators(TRAIN_DIR, VAL_DIR)
    model = cnn_model(input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3))
    model, history = train_cnn_model(model, train_generator, val_generator, epochs=60)

    # Save the trained model to Google Drive
    model_path = '/content/drive/MyDrive/AI/Ai train save path/meat_freshness_classifier.h5'
    model.save(model_path)
    print(f"Model training complete and saved as '{model_path}'.")

In [81]:
# Run the trainer
main()

Found 1815 images belonging to 3 classes.
Found 403 images belonging to 3 classes.


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/60


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


16/16 ━━━━━━━━━━━━━━━━━━━━ 34s 1s/step - accuracy: 0.3540 - loss: 0.3278 - val_accuracy: 0.3945 - val_loss: -4.9590
Epoch 2/60
16/16 ━━━━━━━━━━━━━━━━━━━━ 37s 1s/step - accuracy: 0.3486 - loss: -22.3193 - val_accuracy: 0.3945 - val_loss: -322.1575
Epoch 3/60
16/16 ━━━━━━━━━━━━━━━━━━━━ 29s 1s/step - accuracy: 0.3373 - loss: -438.7069 - val_accuracy: 0.3995 - val_loss: -1721.9427
Epoch 4/60
16/16 ━━━━━━━━━━━━━━━━━━━━ 41s 1s/step - accuracy: 0.3391 - loss: -2371.9355 - val_accuracy: 0.3945 - val_loss: -9895.8730
Epoch 5/60
16/16 ━━━━━━━━━━━━━━━━━━━━ 41s 1s/step - accuracy: 0.3611 - loss: -9478.2197 - val_accuracy: 0.4144 - val_loss: -21476.0586
Epoch 6/60
16/16 ━━━━━━━━━━━━━━━━━━━━ 28s 1s/step - accuracy: 0.3676 - loss: -32869.0234 - val_accuracy: 0.3945 - val_loss: -112761.7109
Epoch 7/60
16/16 ━━━━━━━━━━━━━━━━━━━━ 29s 1s/step - accuracy: 0.3469 - loss: -92894.0547 - val_accuracy: 0.3921 - val_loss: -259446.1875
Epoch 8/60
16/16 ━━━━━━━━━━━━━━━━━━━━ 40s 1s/step - accuracy: 0.3539 - loss: 

Model training complete and saved as '/content/drive/MyDrive/AI/Ai train save path/meat_freshness_classifier.h5'.
